# Question 3 Vision

### Overview:

One of the software division’s primary tasks is identifying and localizing ODLCs as mentioned previously. As written in the reference, these ODLCs are composed of a unique color, character, and shape that must be classified from 100ft in the air. However, in order to train models with high accuracy, we would need to collect thousands of images over 100+ hours of flight time. We, unfortunately, cannot reasonably perform this task before the competition deadline. So, in order to have both well-trained models within timeframes, we use generation scripts which can produce thousands of images within seconds that can somewhat match the quality and precision of the camera in black and white. 

### Your Task:

- Write generator script(s) for shape and/or character.
- Train a YOLOv8n model on this data (hint: is there a way to not manually label every image?).
- Correctly recognize the shape and character of the ODLC.

### Constraints: 

Character - At SUAS, all alphanumeric characters are allowed, but for simplicity, it can be assumed that all test cases are uppercase letters (ABCDEFGHIJKLMNOPQRSTUVWXYZ)
Shape - The only valid shapes are circle, semicircle, quarter circle, triangle, rectangle, pentagon, star, and cross.

### Restrictions:

Don’t use non-standard libraries except PIL, ultralytics, numpy, and opencv-python.

P.S: If it is taking more than 5 hours to train a YOLOv8n model, reduce the number of training images. I am mostly trying to learn about your problem-solving process rather than the actual results.

### File Input Format: 
The .in file will consist of a square image with random dimensions, and the ODLC is guaranteed to be in the image however, it may not be in the center.

### File Output Format: 
Output the shape and character respectively separated by spaces. When submitting the deliverables for this problem, include the YOLOv8 .pt model along with the models, labels, and all YOLOv8 created (like runs and predict) directories.

### Draw a Circle

In [1]:
from PIL import Image, ImageDraw, ImageFont
import random

# fix the image size
image_size = 360

# Draw the shape that fits 70-80% of the image
shape_size = image_size * random.randint(70, 80) // 100


In [2]:
# Draw a circle
def draw_ellipse(draw, center, size):
    draw.ellipse((center[0] - size // 2, center[1] - size // 2, center[0] + size // 2, center[1] + size // 2), outline='black', width=10)
    bbox = (center[0] - size // 2, center[1] - size // 2, center[0] + size // 2, center[1] + size // 2)
    return bbox

In [3]:
# Create a blank image
img = Image.new('RGB', (image_size, image_size), color='white')
draw = ImageDraw.Draw(img)

shape_bb = draw_ellipse(draw, [image_size//2, image_size//2], shape_size)
print("shape bounding box =", shape_bb)
img.show()

shape bounding box = (40, 40, 320, 320)


### Draw a Character

In [4]:
# Draw the character that fits inside the shape
def draw_character(draw, character, bbox, char_ratio = 0.75, font_type = "Arial Unicode"):
    from PIL import ImageFont

    # Choose a font size that fits inside the shape's bounding box
    font_size = min(bbox[2] - bbox[0], bbox[3] - bbox[1]) * char_ratio
    print("font_size = ", font_size)
    font = ImageFont.truetype(f"/Library/Fonts/{font_type}.ttf", font_size)
    
    center_x = bbox[0] + (bbox[2] - bbox[0]) / 2
    center_y = bbox[1] + (bbox[3] - bbox[1]) / 2
    print("center x, y =", center_x, center_y)

    # Calculate text size and position the character inside the shape
    text_len = draw.textlength(character, font=font)
    print("text_length = ", text_len)

    text_bb = draw.textbbox((center_x, center_y), character, font=font)
    print("text_bbox =", text_bb)

    text_height = text_bb[3] - text_bb[1]
    print("text_height =", text_height)

    text_left = center_x - text_len / 2
    text_top = center_y - text_height
    text_bb = draw.textbbox((text_left, text_top), character, font=font)
    print("new text_bbox =", text_bb)

    draw.text((text_left, text_top), character, font=font, fill='black')

    return text_bb


In [5]:
# Create a blank image
img = Image.new('RGB', (image_size, image_size), color='white')
draw = ImageDraw.Draw(img)

circle_bb = draw_ellipse(draw, [image_size//2, image_size//2], shape_size)
print("circle bounding box =", circle_bb)

char_bb = draw_character(draw, "A", circle_bb)
print("text bounding box = ", char_bb)
img.show()

circle bounding box = (40, 40, 320, 320)
font_size =  210.0
center x, y = 180.0 180.0
text_length =  140.0625
text_bbox = (179.0, 255.0, 321.0, 405.0)
text_height = 150.0
new text_bbox = (108.96875, 105.0, 250.96875, 255.0)
text bounding box =  (108.96875, 105.0, 250.96875, 255.0)


### Draw a Triangle

In [6]:
def draw_triangle(draw, center, size):
    points = [(center[0], center[1] - size // 2), (center[0] - size // 2, center[1] + size // 2), (center[0] + size // 2, center[1] + size // 2)]
    draw.polygon(points, outline='black', width=10)
    bbox = (center[0] - size // 2, center[1] - size // 2, center[0] + size // 2, center[1] + size // 2)
    return bbox

In [7]:
# Create a blank image
img = Image.new('RGB', (image_size, image_size), color='white')
draw = ImageDraw.Draw(img)

shape_bb = draw_triangle(draw, [image_size//2, image_size//2], shape_size)
print("shape bounding box =", shape_bb)

char_bb = draw_character(draw, "U", shape_bb, char_ratio=0.6)
print("text bounding box = ", char_bb)
img.show()

shape bounding box = (40, 40, 320, 320)
font_size =  168.0
center x, y = 180.0 180.0
text_length =  121.328125
text_bbox = (180.0, 240.0, 301.0, 362.0)
text_height = 122.0
new text_bbox = (119.3359375, 118.0, 240.3359375, 240.0)
text bounding box =  (119.3359375, 118.0, 240.3359375, 240.0)


### Draw Other Shapes

In [8]:
def draw_semicircle(draw, center, size):
    points = [center[0] - size // 2, center[1] - size // 2, center[0] + size // 2, center[1] + size // 2]
    draw.pieslice(points, 180, 360, outline='black', width=10)
    bbox = (center[0] - size // 2, center[1]- size // 2, center[0] + size // 2, center[1])
    return bbox

In [9]:
def draw_quarter_circle(draw, center, size):
    draw.pieslice([center[0] - size // 2, center[1] - size // 2, center[0] + size // 2, center[1] + size // 2], 270, 360, outline='black', width=10)
    bbox = (center[0], center[1]- size // 2, center[0] + size // 2, center[1])
    return bbox

In [10]:
def draw_rectangle(draw, center, size):
    draw.rectangle((center[0] - size // 2, center[1] - size // 4, center[0] + size // 2, center[1] + size // 4), outline='black', width=10)
    bbox = (center[0] - size // 2, center[1] - size // 4, center[0] + size // 2, center[1] + size // 4)
    return bbox

In [11]:
def draw_pentagon(draw, center, size):
    draw.regular_polygon((center[0], center[1], size//2), n_sides=5, outline="black", width=10)
    bbox = (center[0] - size // 2, center[1] - size // 2, center[0] + size // 2, center[1] + size // 2)
    return bbox

In [12]:
def draw_star(draw, center, size):
    draw.regular_polygon((center[0], center[1], size //2), n_sides=5, rotation=30, outline="black", width=10)
    bbox = (center[0] - size // 2, center[1] - size // 2, center[0] + size // 2, center[1] + size // 2)
    return bbox

In [13]:
def draw_shape(draw, shape, center, size):
    if shape == "circle":
        return draw_ellipse(draw, center, size)
    elif shape == "triangle":
        return draw_triangle(draw, center, size)
    elif shape == "semicircle":
        return draw_semicircle(draw, center, size)
    elif shape == "quarter_circle":
        return draw_quarter_circle(draw, center, size)
    elif shape == "rectangle":
        return draw_rectangle(draw, center, size)
    elif shape == "pentagon":
        return draw_pentagon(draw, center, size)
    elif shape == "star":
        return draw_star(draw, center, size)

In [14]:
def draw_shape_with_letter(shape, letter, text_ratio=0.75):
    # Create a blank image
    img = Image.new('RGB', (image_size, image_size), color='white')
    draw = ImageDraw.Draw(img)

    print("shape =", shape)
    
    shape_bb = draw_shape(draw, shape, [image_size//2, image_size//2], shape_size)
    print("shape bounding box =", shape_bb)

    text_bb = draw_character(draw, letter, shape_bb, char_ratio=text_ratio)
    print("text bounding box = ", text_bb)
    img.show()

In [17]:
# Test with all shapes
shapes = ['circle', 'semicircle', 'quarter_circle', 'triangle', 'rectangle', 'pentagon', 'star', 'cross']
letters = ['A', 'U', 'L', 'J']
for shape in shapes[:-1]:
    for letter in letters:
        draw_shape_with_letter(shape, letter)

shape = circle
shape bounding box = (40, 40, 320, 320)
font_size =  210.0
center x, y = 180.0 180.0
text_length =  140.0625
text_bbox = (179.0, 255.0, 321.0, 405.0)
text_height = 150.0
new text_bbox = (108.96875, 105.0, 250.96875, 255.0)
text bounding box =  (108.96875, 105.0, 250.96875, 255.0)
shape = circle
shape bounding box = (40, 40, 320, 320)
font_size =  210.0
center x, y = 180.0 180.0
text_length =  151.65625
text_bbox = (180.0, 255.0, 332.0, 408.0)
text_height = 153.0
new text_bbox = (104.171875, 102.0, 256.171875, 255.0)
text bounding box =  (104.171875, 102.0, 256.171875, 255.0)
shape = circle
shape bounding box = (40, 40, 320, 320)
font_size =  210.0
center x, y = 180.0 180.0
text_length =  116.796875
text_bbox = (180.0, 255.0, 297.0, 405.0)
text_height = 150.0
new text_bbox = (121.6015625, 105.0, 238.6015625, 255.0)
text bounding box =  (121.6015625, 105.0, 238.6015625, 255.0)
shape = circle
shape bounding box = (40, 40, 320, 320)
font_size =  210.0
center x, y = 180.0 180